In [ ]:
import pandas as pd
import json
import pickle5 as pickle
from tqdm import tqdm
import numpy as np

from plotnine import *
import plotly.express as px

import eval

In [ ]:
# load explanations
results_dir = '../../explanations/linklogic/full_parents_benchmark/'
experiments = {'before_sib': ['fb13', 'False'],
              'after_sib': ['fb14', 'False']}
figure_dir = '../../figures/figures_for_paper/'

In [ ]:
results = dict()
for experiment, strings in experiments.items():
    print(f'Loading results for experiment {experiment}')
    sibling = strings[0]
    child = strings[1]
    with open(f'{results_dir}/{sibling}_child_{child}.pickle', 'rb') as f:
        results[experiment] = pickle.load(f)

In [ ]:
# load benchmark
with open('../../data/commonsense_benchmark/v4/commonsense_benchmark_all.json', 'r') as f:
    benchmark = json.load(f)
    
print(f'Loaded benchmark with {len(benchmark)} triples')

In [ ]:
thresh_95 = -np.log(0.05)
thresh_90 = -np.log(0.1)

In [ ]:
# extract fidelity for each query triple and generate scatterplot
rows = []
for i, bmk in tqdm(enumerate(benchmark), total=len(benchmark)):
    triple = bmk['query_triple']
    for experiment in experiments.keys():
        res = eval.get_results_for_query_triple(results[experiment], triple)
        
        if not res:
            #print(f'Did not find results for {triple}')
            continue
            
        num_heuristic_paths_90 = sum([1 for f in res['linklogic_features'] if f['kge_score']['path_score'] > thresh_90])
        num_heuristic_paths_95 = sum([1 for f in res['linklogic_features'] if f['kge_score']['path_score'] > thresh_95])
        
        rows.append({
            'experiment': experiment,
            'query_triple': '--'.join(triple),
            'fidelity': res['linklogic_metrics']['test_acc'],
            'num_siblings': len(bmk['entity_names']['siblings']),
            'num_paths_linklogic': len(res['linklogic_explanations']),
            'num_paths_heuristic_90': num_heuristic_paths_90,
            'num_paths_heuristic_95': num_heuristic_paths_95
        })
sibling_metrics = pd.DataFrame(rows)

In [ ]:
sibs = sibling_metrics.pivot(columns='experiment', index=['query_triple', 'num_siblings'], values='fidelity').reset_index()
sibs['any_siblings'] = sibs['num_siblings'] >0
ggplot(sibs, aes(x='before_sib', fill='num_siblings', group='num_siblings')) + geom_density(alpha=0.1) + labs(x='fidelity')

In [ ]:
ggplot(sibs, aes(x='before_sib', y='after_sib')) + geom_point(alpha=0.1, size=3) + geom_abline(slope=1, intercept=0) + facet_grid('.~any_siblings')

In [ ]:
import plotly.express as px

sib1 = sibling_metrics[sibling_metrics.experiment == 'after_sib']
sib1[['num_paths_heuristic_90', 'num_paths_heuristic_95', 'num_paths_linklogic']].hist()
num_paths = sib1.melt(id_vars=['query_triple', 'num_siblings'], value_vars=['num_paths_linklogic', 'num_paths_heuristic_90', 'num_paths_heuristic_95'])
num_paths['variable'] = num_paths['variable'].map({'num_paths_linklogic': 'linklogic', 
                           'num_paths_heuristic_90': 'PathScore, 0.90 threshold',
                           'num_paths_heuristic_95': 'PathScore, 0.95 threshold'})
p = px.box(num_paths, x='num_siblings', y='value', color='variable',
       labels={'num_siblings': 'Number of Siblings', 'value': 'Explanation Size (# of Paths)'})
p.update_layout({'legend_title_text': ''})
p.write_image(f'{figure_dir}/num_paths_by_num_siblings.png')
#p.update_layout(legend_orientation='v')

In [ ]:
ggplot(sib_b4, aes(x='num_siblings', y='num_heuristic_paths', fill='num_siblings', group='num_siblings')) + geom_boxplot(alpha=0.5) #+ geom_abline(slope=1, intercept=0)

In [ ]:
ggplot(sib_b4, aes(x='num_siblings', y='num_linklogic_paths', fill='num_siblings', group='num_siblings')) + geom_boxplot(alpha=0.5)

In [ ]:
# sib_pivot['difference'] = sib_pivot['after_sib'] - sib_pivot['before_sib']
# sib_pivot['dummy'] = 1
# ggplot(sib_pivot, aes(y='difference', x='dummy')) + geom_boxplot()